In [21]:
%matplotlib notebook
# !pip install python-xmp-toolkit

#TODO gebruiker toevoegen, CSV apart opslaan  !! CHECK
#LEFT/RIGHT marker toevoegen CHECK
#123456789 voor middelpunten (toevoegen via + knop)
#Beeldbewerking

#EXIF/XMP readout disabled for mybinder environment
#from libxmp import XMPFiles, consts

#use matplotlib notebook in the mybinder-environment and ipympl local
    
#setup definitions
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import rand
import matplotlib.image as mpimg
import matplotlib.patches as patches
import os
import csv
#some globals
filename=''
#altitude=0
#groundscale=1
ShowCSV=False

#make a sorted list of available JPG starting with DJI
jpegs = sorted([f for f in os.listdir('./') if (f.endswith('.JPG')) and f.startswith('DJI_')] ) 

#setup matplotlib
fig = plt.figure()

ax = fig.add_subplot(111)
ax.axis('equal')
def length(p1,p2):
    return np.sqrt((p1.point.center[0]-p2.point.center[0])**2+(p1.point.center[1]-p2.point.center[1])**2)

#routine to calculate some info
def calculatelength():
     global altitude
     global groundscale
     len1=length(drs[0],drs[2])
     len2=length(drs[0],drs[1])+length(drs[1],drs[2])
     width=drs[1].point.radius*2
     width2=length(drs[3],drs[4])
     try:
         ratio1=len1/(width)
         ratio2=len2/(width)
         ax.set_title("L1:%dpx L2:%dpx W1:%dpx W2:%dpx R1 %0.1F R2 %0.1F "  % ( len1,len2, width,width2,ratio1,ratio2))   
         #est_len=len1*groundscale   
         #ax.set_title("Alti:%dm  L1:%dpx L2:%dpx W1:%dpx W2:%dpx R1 %0.1F R2 %0.1F \n length %0.1F(cm)"  % (altitude, len1,len2, width,width2,ratio1,ratio2,est_len))
     except:
        pass
        #ratio not calculated
        #ax.set_xlabel(filename)
    
# read a JPG file into the plot
def usefile(fname):
    global filename
    filename=(fname)
    
    global img
    #expected markers-filename
    filecsv=filename.split('.')[0]+ '.csv'

    img_orig = mpimg.imread(filename)    
    img=img_orig
    #DISABLED !!!! for mybinder environment
    #retrieve XMP data (altitude etc)
    #xmpfile = XMPFiles( file_path=filename, open_forupdate=True )
    #xmp = xmpfile.get_xmp()
    #global altitude
    #altitude=float(xmp.get_property('http://www.dji.com/drone-dji/1.0/','RelativeAltitude'))
    
    #calculate resolution based on altitude in cm/pix
    #phantom 4pro Focallength 8.8 (sensorsize 13.2 x 8.8 mm), alleen breedte gebruiken !!
    #imagesize default 4864 sensorwidth 13.2 
    #calc (sensorW * altitude *100)/ (focalL*imageW)
    #global groundscale
    #groundscale=(13.2*altitude*100)/(8.8*4864)
    
    ax.imshow(img)
    #axis([xmin,xmax,ymax,ymin]) reverse YAXIS !!
    ax.axis([0,img.shape[1],img.shape[0],1])
    global ShowCSV
    #check if a file with markers was saved before
    try:
      with open(filecsv) as csv_file:
        csv_reader = csv.reader(csv_file)
        #reader = csv.reader(infile)
        next(csv_reader, None)  # skip the headers
        xmin=10000
        xmax=-1
        ymin=10000
        ymax=-1
        count=0
        for circ in circles: 
            csvdata=next(csv_reader,None)
            csv_x=float(csvdata[0])
            csv_y=float(csvdata[1])
            if ShowCSV==True:
                circ.center=float(csv_x), float(csv_y)
                try:
                  circ.radius=float(csvdata[2])  
                except:
                  pass
                try:
                  circ.set_label(csvdata[3])
                except:
                  pass
            if float(csv_x)>xmax:
                xmax=csv_x
            if float(csv_y)>ymax:
                ymax=csv_y
            if float(csv_x)<xmin:
                xmin=csv_x
            if float(csv_y)<ymin:
                ymin=csv_y
            count+=1

        #zoom to area around nose/middle/tail markers          
        #plt.axis([xmin*0.8,xmax*1.2,ymax*1.1,ymin*0.9])
        ax.axis([xmin-150,xmax+150,ymax+100,ymin-100])

        calculatelength()

    except:
         print('no markers found')
         ax.set_xlabel('set markers, press N(nose), M(middle) or T(tail) at position \n use cursor up/dn to change markersize')

#DUMMY routine to capture events on the canvas
class DraggablePoint:
    lock = None #only one can be animated at a time
    def __init__(self, point):
        self.point = point
        self.press = None
        self.background = None

#create labeled circles         
circles = [patches.Circle((120,10), 10, ec='red', fc='none', alpha=1, lw=2, ls='solid', label='N' ),
           patches.Circle((120,30), 10, ec='white', fc='none', alpha=1, lw=2, ls='solid', label='M'),
           patches.Circle((120,50), 10, ec='lightgreen', fc='none', alpha=1, lw=2, ls='solid', label='T'),
           patches.Circle((120,70), 10, ec='yellow', fc='none', alpha=1, lw=2, ls='solid', label='L'),
           patches.Circle((120,90), 10, ec='yellow', fc='none', alpha=1, lw=2, ls='solid', label='R')
          ]
#add circles to the plot
drs = []  
for circ in circles:
      ax.add_patch(circ)
      dr = DraggablePoint(circ)
      drs.append(dr)
        
#CANVAS EVENTS    ************************************************************************
def on_pick (event):
    #ax.set_xlabel('pressed: %s' %(event.key))
    if event.key=='n': # position nose
            drs[0].point.center=event.xdata, event.ydata
            fig.canvas.draw()
    if event.key=='m': # position fin
            drs[1].point.center=event.xdata, event.ydata
            fig.canvas.draw()
    if event.key=='t': # position notch-tail
            drs[2].point.center=event.xdata, event.ydata
            fig.canvas.draw()
    if event.key=='l': # position notch-tail
            drs[3].point.center=event.xdata, event.ydata
            fig.canvas.draw()
    if event.key=='r': # position notch-tail
            drs[4].point.center=event.xdata, event.ydata
            fig.canvas.draw()
        
    larger_keys=['up','right','pgup']
    smaller_keys=['down','left','pgdn']
    #print('you pressed', event.key, event.xdata, event.ydata, self.point.center[0], self.point.center[1])
    #only allow the middle to be size adjusted
    if event.key in larger_keys: 
        drs[1].point.radius+=1
    if event.key in smaller_keys: 
        drs[1].point.radius-=1
        if drs[1].point.radius<5:
            drs[1].point.radius=5
    calculatelength()        

#not used            
def on_motion (event):    
    #tx = ' xdata=%f, ydata=%f' % (event.xdata, event.ydata)
    #ax.set_xlabel(tx)
    pass
    
def onclick(event):
    global filename
    #tx = ' button=%d, x=%d, y=%d, xdata=%f, ydata=%f' % (event.button, event.x, event.y, event.xdata, event.ydata)
    tx = ' file=%s,  xdata=%f, ydata=%f' % (filename, event.xdata, event.ydata)
    #text.set_text(tx)
    ax.set_xlabel(tx)
    #fig(title=tx)

kpres=fig.canvas.mpl_connect('key_press_event',on_pick)
cid = fig.canvas.mpl_connect('button_press_event', onclick)
#motion = fig.canvas.mpl_connect('motion_notify_event', on_motion)

#GUI DEFS    ************************************************************************
SaveCSVButton = widgets.Button(
    description='Save CSV',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save CSV',
    #icon='arrows-alt'
)

def saveCSV(new):
        'save data to CSV as X,Y,radius'
        filecsv=jpegselectie.value
        if Username.value.lower()=='inger': #default INGER 
          filecsv=filecsv.split('.')[0]+ '.csv'
        else:
          filecsv=filecsv.split('.')[0]+ '_user_'+Username.value+'.csv'  
        with open(filecsv, mode='w') as markers_file:
            fieldnames = ['x', 'y', 'radius', 'label']
            writer = csv.DictWriter(markers_file, fieldnames=fieldnames)
            writer.writeheader()
            for circ in circles:
               labl=circ.get_label()
               writer.writerow({'x':circ.center[0],'y':circ.center[1], 'radius': circ.radius, 'label':labl})
    
SaveCSVButton.on_click(saveCSV)

Username=widgets.Text(
    value='',
    placeholder='Type your name',
    description='String:',
    disabled=False
)

def userchange(new):
    global ShowCSV
    if (Username.value).lower()=='inger':
        ShowCSV=True
    else:
        ShowCSV=False

Username.observe(userchange, names="value")

ResetZoomButton = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reset zoom',
    icon='arrows-alt'
)
def resetZoom(new):
    # Reset the x and y axes on the figure
    ax.axis([0,img.shape[1],img.shape[0],1])
    
ResetZoomButton.on_click(resetZoom)

jpegselectie = widgets.Dropdown(
    options=jpegs,
    value=jpegs[0],
    description='JPEG:',
)

def fileselect(change):
    usefile(jpegselectie.value)
    
jpegselectie.observe(fileselect, names="value")
    
#OUTPUT 

display(widgets.HBox([Username, jpegselectie,ResetZoomButton, SaveCSVButton]))
leg = ax.legend(framealpha=0.2, facecolor='white')
plt.setp(leg.get_texts(), color='w')
usefile(jpegs[0])



FigureCanvasNbAgg()

no markers found
